<a href="https://colab.research.google.com/github/vikasbda21/PyTorch-for-DL/blob/main/NN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#set device

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print("device :",device)

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 1

# load Data

train_dataset  = datasets.MNIST(root ='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root ='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader =  DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# Loss and Optimizer
optimizer = optim.Adam(model.parameters(), lr= learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epoch):
    for batch_idx ,(data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        data = data.reshape(data.shape[0], -1)

        scores = model(data)
        loss = criterion(scores, targets)


        optimizer.zero_grad()
        loss.backward()

        optimizer.step()



def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training dataset")
    else:
        print("Checking accuracy on testing dataset")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples}  with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()
    return num_correct / num_samples

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


device : cuda
Checking accuracy on training dataset
Got 56041 / 60000  with accuracy 93.40
Checking accuracy on testing dataset
Got 9331 / 10000  with accuracy 93.31


tensor(0.9331, device='cuda:0')